<a href="https://colab.research.google.com/github/comp-neural-circuits/plasticity-workshop/blob/dev/hebbian_plasticity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys, json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.stats import norm

In [3]:
# utility functions

def setup_simulation(params):
    t = np.arange(0., params['t_sim'], params['dt'])
    rX = np.zeros((params['n0'], len(t)))
    rY = np.zeros((params['n1'], len(t)))
    w01 = np.zeros((params['n0']*params['n1'], len(t))) # later modify to store weights only sdt time steps
    return [t, rX, rY, w01]

def run_simulation(params):
    
    [t, rX, rY, w01] = setup_simulation(params)
    
    
    for _i, _t in t:
        
        # state at time _t
        rx0, ry0 = rX[:,_i], rY[:,_i]
        wxy0 = wXY[:,_i]
        
        # specify input at this time and modify rx
        #rx1 = 
        
        # update state to time _t + 1
        ry1 = ry0 + (dt/taum) * (-ry0 + np.dot(rx0, wxy0.reshape(params['n0'], params['n1'])))
        wxy1 = update_weights(wxy0, rx1, ry1, params)
        
        # store
        rX[:,_i+1], rY[:,_i+1] = rx1, ry1
        wXY[:,_i+1] = wxy1
        
    
    
    return rX, rY, wXY
    

In [4]:
# plasticity functions

def update_weights_hebbian(w0, r0, r1, params):
    """
    r0 = rates of presynatic neurons   (n,1)
    r1 = rates of postsynaptic neurons (n,1)
    w0 = initial synaptic weights      (n,n)
    w1 = final synaptic weights        (n,n)
    """
    w1 = w0 + (params['dt']/params['tauw']) * np.dot(r1.reshape(params['n1'],1), r0.reshape(1,params['n0'])).reshape(-1)
    w1 = rescale_weights(w1, params['w_low'], params['w_high'])
    return w1

# there can also be hebbian with threshold (which then uses input covariance matrix instead of correlation matrix)

def update_weights_hebbian_normalized(w0):
    return w1

def update_weights_BCM(w0):
    return w1

def rescale_weights(w, lower_bound, upper_bound):
    w[w < lower_bound] = lower_bound
    w[w > upper_bound] = upper_bound
    return w

In [6]:
# input

def generate_regular_input_times(inter_event_interval, t_sim):
    return np.arange(0, t_sim, inter_event_interval)

# input: L-events

def generate_L_events_times(t_end, inter_event_interval):
    # generate sequence of L-event start times
    L_events_times = []
    _pt = 0. 
    while _pt < t_end:
        _pt += np.random.exponential(inter_event_interval)
        L_events_times.append(round(_pt, 1))
    return L_events_times

def define_L_event(n, amp_L, mean_dur, pct_low, pct_high):
    # for one L-event, specify the L-amp, L-neurons, L-dur
    amp = amp_L                                                             # amplitude
    dur = round(np.random.normal(mean_dur, mean_dur/10.), 1)                # duration
    pct = round(np.random.uniform(low=pct_low, high=pct_high), 2)           # participation ratio (20-60%)
    sN = np.random.choice(np.arange(n), 1)
    neur = np.roll(np.arange(n), -sN)[:int(pct*n)]                          # stimulated neurons
    return amp, dur, pct, neur

In [ ]:
# define parameters

params = {
    'n0' = 10,              # number of presynaptic neurons
    'n1' = 10,              # number of postsynaptic neurons
    'taum' = 10.,           # tau for rate equation (ms)
    'tauw' = 100000.,       # tau for synaptic weight equation (ms)
    't_sim' = 5000.,        # Time of simulation (ms)    
    'dt' = 0.1,             # Time step for simulation
    'sdt' = 100.,           # Time step for storing/recording weights
    'input_IEI' = 100.      # inter event interval for input events
}

Task: To form Receptive Fields or Orientation selective neurons

In [ ]:
# main

input_times = generate_regular_input_times(params['input_frequency'], params['t_sim'])


